In [1]:
from symred.chem import xyz_from_pubchem
import numpy as np
from openfermion import FermionOperator
from scipy.sparse.linalg import expm
from openfermion import get_sparse_operator

In [2]:
mol_name = 'H2O'

In [3]:
xyz_file = xyz_from_pubchem(mol_name)

In [4]:
# xyz_file = '4\n \nH\t0\t0\t0\nH\t1\t0\t0\nH\t2\t0\t0\nH\t3\t0\t0'
# print(xyz_file)


xyz_file = '4\n \nH\t0\t0\t0\nH\t2.45366053071732\t0\t0\nH\t2.45366053071732\t2.45366053071732\t0\nH\t0\t2.45366053071732\t0'
# geometry=[
#     ('H', (0.0,0.0,0.0)),
#     ('H', (2.45366053071732,0.0,0.0)),
#     ('H', (2.45366053071732,2.45366053071732,0.0)),
#     ('H', (0.0,2.45366053071732,0.0))
#      ]  
print(xyz_file)

4
 
H	0	0	0
H	2.45366053071732	0	0
H	2.45366053071732	2.45366053071732	0
H	0	2.45366053071732	0


In [5]:
from symred.chem import PySCFDriver

In [6]:
basis = 'STO-3G'
convergence = 1e6
charge=0
max_hf_cycles=1000
ram = 8_000
run_mp2  = False,
run_cisd = False,
run_ccsd = True,
run_fci  = False

In [7]:
pyscf_obj = PySCFDriver(xyz_file,
                       basis,
                       convergence=convergence,
                       charge=charge,
                       max_ram_memory=ram,
                       max_hf_cycles=max_hf_cycles,
                       
                       run_mp2=run_mp2,
                       run_cisd=run_cisd,
                       run_ccsd=run_ccsd,
                       run_fci=run_fci)

In [8]:
pyscf_obj.run_pyscf()

/Users/lex/anaconda3/envs/symred/lib/python3.9/site-packages/pyscf/cc/ccsd.py:232: RuntimeWarning: invalid value encountered in add
  t2new[:,:,q0:q1] += lib.einsum('kica,ckjb->ijab', theta, wVOov)
/Users/lex/anaconda3/envs/symred/lib/python3.9/site-packages/pyscf/cc/ccsd.py:710: RuntimeWarning: invalid value encountered in add
  tau = t2[:,:,p0:p1] + numpy.einsum('ia,jb->ijab', t1[:,p0:p1], t1)
/Users/lex/anaconda3/envs/symred/lib/python3.9/site-packages/pyscf/cc/ccsd.py:393: RuntimeWarning: invalid value encountered in add
  tau[p0:p1] += t2[i,:i+1]
/Users/lex/anaconda3/envs/symred/lib/python3.9/site-packages/pyscf/cc/ccsd.py:327: RuntimeWarning: invalid value encountered in add
  tau = t2[i,:,p0:p1] + numpy.einsum('a,jb->jab', t1[i,p0:p1], t1)
/Users/lex/anaconda3/envs/symred/lib/python3.9/site-packages/pyscf/cc/ccsd.py:335: RuntimeWarning: invalid value encountered in subtract
  theta -= t2[:,:,p0:p1].transpose(0,2,1,3)
/Users/lex/anaconda3/envs/symred/lib/python3.9/site-packages/p

In [9]:
2*pyscf_obj.pyscf_hf.mol.nao

8

In [10]:
from symred.chem import FermionicHamilt

In [11]:
H_ferm = FermionicHamilt(pyscf_obj.pyscf_hf)

H_ferm.build_operator()

H_ferm.fermionic_molecular_hamiltonian

() 1.1676751516332355
((0, 1), (0, 0)) -1.0940851751125964
((0, 1), (6, 0)) -0.02524310586380466
((1, 1), (1, 0)) -1.0940851751125964
((1, 1), (7, 0)) -0.02524310586380466
((2, 1), (2, 0)) -1.045835726765513
((3, 1), (3, 0)) -1.045835726765513
((4, 1), (4, 0)) -1.0458357267655132
((5, 1), (5, 0)) -1.0458357267655132
((6, 1), (0, 0)) -0.02524310586380467
((6, 1), (6, 0)) -1.004747518161057
((7, 1), (1, 0)) -0.02524310586380467
((7, 1), (7, 0)) -1.004747518161057
((0, 1), (0, 1), (0, 0), (0, 0)) 0.18183664435793204
((0, 1), (0, 1), (0, 0), (6, 0)) -0.027369665234424485
((0, 1), (0, 1), (2, 0), (2, 0)) 0.11456979763440035
((0, 1), (0, 1), (4, 0), (4, 0)) 0.037803923014085604
((0, 1), (0, 1), (6, 0), (0, 0)) -0.02736966523442448
((0, 1), (0, 1), (6, 0), (6, 0)) 0.0470668225562559
((0, 1), (1, 1), (1, 0), (0, 0)) 0.18183664435793204
((0, 1), (1, 1), (1, 0), (6, 0)) -0.027369665234424485
((0, 1), (1, 1), (3, 0), (2, 0)) 0.11456979763440035
((0, 1), (1, 1), (5, 0), (4, 0)) 0.03780392301408560

In [12]:
hf_state = H_ferm.hf_comp_basis_state
hf_state


array([1, 1, 1, 1, 0, 0, 0, 0])

In [13]:
hf_ket = H_ferm.hf_ket
H_mat = H_ferm.get_sparse_ham()

In [14]:
from pyscf.cc.addons import spatial2spin

# doubles!

t2 = spatial2spin(pyscf_obj.pyscf_ccsd.t2)
no, nv = t2.shape[1:3]
nmo = no + nv
double_amps = np.zeros((nmo, nmo, nmo, nmo))
double_amps[no:,:no,no:,:no] = .5 * t2.transpose(2,0,3,1)

In [15]:
# singles



t1 = spatial2spin(pyscf_obj.pyscf_ccsd.t1)
no, nv = t1.shape
nmo = no + nv
ccsd_single_amps = np.zeros((nmo, nmo))
ccsd_single_amps[no:,:no] = t1.T
ccsd_single_amps.shape

(8, 8)

In [16]:
double_amplitudes_list=[]
double_amplitudes = double_amps
for i, j, k, l in zip(*double_amplitudes.nonzero()):
    if not np.isclose(double_amplitudes[i, j, k, l], 0):
        double_amplitudes_list.append([[i, j, k, l],
                                       double_amplitudes[i, j, k, l]])
    
double_amplitudes_list

[[[4, 0, 4, 0], nan],
 [[4, 0, 4, 2], nan],
 [[4, 0, 5, 1], nan],
 [[4, 0, 5, 3], nan],
 [[4, 0, 6, 0], nan],
 [[4, 0, 6, 2], nan],
 [[4, 0, 7, 1], nan],
 [[4, 0, 7, 3], nan],
 [[4, 1, 5, 0], nan],
 [[4, 1, 5, 2], nan],
 [[4, 1, 7, 0], nan],
 [[4, 1, 7, 2], nan],
 [[4, 2, 4, 0], nan],
 [[4, 2, 4, 2], nan],
 [[4, 2, 5, 1], nan],
 [[4, 2, 5, 3], nan],
 [[4, 2, 6, 0], nan],
 [[4, 2, 6, 2], nan],
 [[4, 2, 7, 1], nan],
 [[4, 2, 7, 3], nan],
 [[4, 3, 5, 0], nan],
 [[4, 3, 5, 2], nan],
 [[4, 3, 7, 0], nan],
 [[4, 3, 7, 2], nan],
 [[5, 0, 4, 1], nan],
 [[5, 0, 4, 3], nan],
 [[5, 0, 6, 1], nan],
 [[5, 0, 6, 3], nan],
 [[5, 1, 4, 0], nan],
 [[5, 1, 4, 2], nan],
 [[5, 1, 5, 1], nan],
 [[5, 1, 5, 3], nan],
 [[5, 1, 6, 0], nan],
 [[5, 1, 6, 2], nan],
 [[5, 1, 7, 1], nan],
 [[5, 1, 7, 3], nan],
 [[5, 2, 4, 1], nan],
 [[5, 2, 4, 3], nan],
 [[5, 2, 6, 1], nan],
 [[5, 2, 6, 3], nan],
 [[5, 3, 4, 0], nan],
 [[5, 3, 4, 2], nan],
 [[5, 3, 5, 1], nan],
 [[5, 3, 5, 3], nan],
 [[5, 3, 6, 0], nan],
 [[5, 3, 6

In [17]:
single_amplitudes_list = []
for i, j in zip(*ccsd_single_amps.nonzero()):
    single_amplitudes_list.append([[i, j], ccsd_single_amps[i, j]])

single_amplitudes_list

[[[4, 0], nan],
 [[4, 2], nan],
 [[5, 1], nan],
 [[5, 3], nan],
 [[6, 0], nan],
 [[6, 2], nan],
 [[7, 1], nan],
 [[7, 3], nan]]

In [18]:
generator_t2 = FermionOperator()

# Add double excitations
for (i, j, k, l), t_ijkl in double_amplitudes_list:
    i, j, k, l = int(i), int(j), int(k), int(l)
    generator_t2 += FermionOperator(((i, 1), (j, 0), (k, 1), (l, 0)), t_ijkl)
#     if anti_hermitian:
#         generator += FermionOperator(((l, 1), (k, 0), (j, 1), (i, 0)),
#                                      -t_ijkl)

In [19]:
generator_t1 = FermionOperator()
for (i, j), t_ij in single_amplitudes_list:
    i, j = int(i), int(j)
    generator_t1 += FermionOperator(((i, 1), (j, 0)), t_ij)

In [20]:
T = generator_t1 + 0.5*generator_t2

ccsd_ansatz = expm(get_sparse_operator(T, n_qubits=H_ferm.n_qubits))

ValueError: cannot convert float NaN to integer

In [ ]:
ccsd_ket =  ccsd_ansatz @ hf_ket
hf_ket.conj().T @ H_mat @ ccsd_ket

In [ ]:
pyscf_obj.pyscf_ccsd.e_tot

In [ ]:
for ind, amp in enumerate(ccsd_ket):
    if not np.isclose(amp, 0):
        binary_string = np.binary_repr(ind, width=H_ferm.n_qubits)
        print(f'{amp}|{binary_string}>')

In [ ]:
red_ket = ccsd_ket - hf_ket

ccsd_ket @ H_mat @ ccsd_ket